# Genrating a random stabilizer ensemble


**Goal.**  
To generate a random *stabilizer ensemble* by constructing random **Clifford circuits** on $n$ qubits and applying them to the trivial stabilizer state $|0\rangle^{\otimes n}$.  

Each Clifford circuit maps stabilizer states to stabilizer states, so the output of a random Clifford on $|0\rangle^{\otimes n}$ forms a random stabilizer state.  

**Concept.**  
The Clifford group $\mathcal{C}_n$ is the normalizer of the $n$-qubit Pauli group $\mathcal{P}_n$ under conjugation.  
That is,  
$$
U P U^\dagger \in \mathcal{P}_n, \quad \forall\, U \in \mathcal{C}_n,\, P \in \mathcal{P}_n.
$$  
Hence, applying $U \in \mathcal{C}_n$ to the stabilizer state $|0\rangle^{\otimes n}$ preserves the stabilizer structure while randomizing its orientation in Hilbert space.

**Outline.**
1. Choose qubit number $n$ and circuit depth $d$.
2. Construct a random Clifford circuit from gates $\{H, S, X, Y, Z, \mathrm{CX}\}$.
3. Evolve $|0\rangle^{\otimes n}$ under that circuit to obtain the final statevector.
4. Derive its stabilizer group by conjugating the $Z_i$ generators through the Clifford Tableau.
5. Save the state and stabilizer generators to text files for later MPS or entanglement analysis.



## Constructing a random Clifford circuit and getting the statevector

**Purpose.**  
This block builds a random $n$-qubit Clifford circuit, simulates its evolution on the initial state $|0\rangle^{\otimes n}$, and stores the resulting statevector amplitudes.

**Steps.**

- **Imports:**  
  `QuantumCircuit` constructs circuits and `Statevector` simulates pure states.

- **Parameters:**  
  $n$ — number of qubits,  
  $d$ — circuit depth (number of layers).

- **Circuit construction:**  
  For each layer $l = 1, \dots, d$:
  - Apply a random single-qubit Clifford gate chosen from $\{H, X, Y, Z, S\}$ to each qubit.  
  - Add entanglement by selecting random pairs $(q_1, q_2)$ and applying controlled-$X$ ($\mathrm{CX}$) gates.

- **Simulation:**  
  Initialize $|\psi_0\rangle = |0\rangle^{\otimes n}$ and evolve:  
  $$
  |\psi_f\rangle = U_{\text{Clifford}}\,|\psi_0\rangle.
  $$

- **Storage:**  
  Write the real and imaginary parts of each amplitude of $|\psi_f\rangle$ to a text file for later use.

**Key ideas.**
- The gate set $\{H, S, X, Y, Z, \mathrm{CX}\}$ generates the Clifford group.  
- The circuit diagram printed by `print(qc)` visualizes the randomly chosen operations.  
- Saving amplitudes allows later reconstruction or MPS decomposition.



In [ ]:
import random
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
dir = "/Users/arya/Documents/Python/DATA/"


#number of qubits
n = 10
#number of layers
depth = 10
#the initial quantum circuit
qc= QuantumCircuit(n)
#Building a random quantum circuit
for i in range(depth):
    for j in range(n):
        gate = random.choice(['h', 'x', 'y', 'z', 's'])
        if gate == 'h':
            qc.h(j)
        elif gate == 'x':
            qc.x(j)
        elif gate == 'y':
            qc.y(j)
        elif gate == 'z':
            qc.z(j)
        elif gate == 's':
            qc.s(j)

    for q in range(n//2):
        q1, q2 = random.sample(range(n), 2)
        qc.cx(q1, q2)

print (qc)

#initial state
initial_state = Statevector.from_label("0" * n)
print ("Initial state:", initial_state)
final_state = initial_state.evolve(qc)
print ("Final state:", final_state)


# stroring the final state in a text file
cl_file = dir + "/cl.txt"
with open(cl_file, "w") as f:
    for amp in final_state.data:
        real = amp.real
        imag = amp.imag
        f.write(f"{real:.6f} + {imag:.6f}im\n")

     ┌───┐┌───┐┌───┐┌───┐                                                  »
q_0: ┤ Y ├┤ H ├┤ S ├┤ H ├──────────────────────────────────────────────────»
     ├───┤├───┤├───┤└───┘     ┌───┐┌───┐          ┌───┐┌───┐┌───┐┌───┐     »
q_1: ┤ S ├┤ X ├┤ S ├──────────┤ X ├┤ X ├──────────┤ X ├┤ Y ├┤ H ├┤ H ├─────»
     ├───┤└─┬─┘└───┘          └─┬─┘├───┤┌───┐┌───┐└─┬─┘├───┤└───┘└───┘     »
q_2: ┤ H ├──┼───────────────────┼──┤ X ├┤ Y ├┤ S ├──■──┤ H ├───────────────»
     ├───┤  │            ┌───┐  │  └─┬─┘└───┘└───┘     ├───┤┌───┐          »
q_3: ┤ Z ├──┼─────────■──┤ S ├──┼────┼────■────────────┤ X ├┤ Z ├──────────»
     ├───┤  │  ┌───┐  │  └───┘  │    │  ┌─┴─┐┌───┐     └─┬─┘└───┘          »
q_4: ┤ H ├──┼──┤ X ├──┼─────────┼────┼──┤ X ├┤ S ├───────┼─────────────────»
     ├───┤  │  ├───┤  │         │    │  └───┘└───┘       │  ┌───┐┌───┐     »
q_5: ┤ S ├──■──┤ Z ├──┼─────────┼────┼─────────■────■────┼──┤ Y ├┤ S ├─────»
     ├───┤┌───┐└───┘  │         │    │  ┌───┐  │    │    │  └───┘├───┤┌───┐»

## Generating the Stabilizer group for the constructed state

## Generating the stabilizer group for the constructed state

**Objective.**  
Compute and save the full **stabilizer group** corresponding to the random stabilizer state produced by the Clifford circuit.

**Procedure.**
1. **Convert circuit to Clifford tableau:**  
   `Clifford(qc)` gives the tableau representation $C$ acting on Paulis via conjugation.

2. **Initialize stabilizers for the zero state:**  
   The state $|0\rangle^{\otimes n}$ is stabilized by  
   $\{ Z_1, Z_2, \ldots, Z_n \}$.  
   The helper function `zero_state_stabilizers(n)` builds this set as a `PauliList`.

3. **Evolve stabilizers:**  
   Conjugate by $C$ to obtain the stabilizers of the final state:  
   $$
   P'_i = C\, P_i\, C^\dagger , \qquad P_i \in \{Z_1,\dots,Z_n\}.
   $$  
   In Qiskit this is `S.evolve(C, frame='conjugate')`.

4. **Export:**  
   Convert each stabilizer to a Pauli-string label (e.g., `"XZZIY..."`) using `to_labels()`  
   and save them to `P_labels.txt`.

**Why this works.**  
A stabilizer state is the unique simultaneous $+1$ eigenstate of an abelian subgroup  
$\mathcal{S} \subset \mathcal{P}_n$ with $n$ independent generators.  
Conjugating by any Clifford $C$ preserves commutation relations:  
$$
\mathcal{S}' = C\,\mathcal{S}\,C^\dagger ,
$$  
so the new set $\mathcal{S}'$ describes the stabilizer group of the evolved state.

**Outputs.**
- `cl.txt` — complex amplitudes of the final statevector.  
- `P_labels.txt` — list of Pauli-string stabilizers.  

These outputs fully specify the random stabilizer state and can be used in subsequent tensor-network or magic-resource analyses.


In [ ]:
from qiskit.quantum_info import Clifford
from qiskit.quantum_info import PauliList

# Getting the cliffords operator Tableau form
C = Clifford(qc)

# Stabilizer Pauli Strings for zero state
def zero_state_stabilizers(n):
    #Creating all the binary combinations of length n
    combos = np.array(np.meshgrid(*[[0, 1]] * n)).T.reshape(-1, n)

    #Making the pauli_string equivalent
    stabilizers = []
    for c in combos:
        pauli_str = ''.join(['Z' if bit == 1 else 'I' for bit in c])
        stabilizers.append(pauli_str)

    return PauliList(stabilizers)

S = zero_state_stabilizers(10)

# Getting the stabilizer group for the final state
P = S.evolve(C, frame='conjugate')
labels = P.to_labels()  
# Saving the stabilizer labels to a text file
with open("/Users/arya/Documents/Python/DATA/P_labels.txt", "w") as f:
    for s in labels:
        f.write(s + "\n")
print(P.to_labels()[:10])
print("Total:", len(P))

Clifford(array([[False, False,  True, False, False, False,  True, False,  True,
         True, False, False, False, False,  True,  True, False, False,
         True,  True, False],
       [ True,  True, False, False, False,  True, False,  True, False,
        False,  True,  True,  True, False, False,  True, False,  True,
         True,  True,  True],
       [False, False,  True,  True, False,  True, False,  True, False,
        False, False,  True,  True, False, False,  True,  True,  True,
         True, False, False],
       [False, False,  True, False, False,  True,  True, False, False,
         True,  True,  True, False, False, False,  True,  True, False,
        False,  True,  True],
       [False, False, False, False,  True, False, False, False, False,
        False, False,  True, False,  True,  True,  True, False, False,
        False,  True, False],
       [ True, False,  True,  True, False,  True,  True, False,  True,
         True, False,  True,  True,  True,  True, False,  Tr